In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df=df.dropna()

In [6]:
X=df.drop('label',axis=1)

In [7]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [8]:
y=df['label']

In [9]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense,Bidirectional

In [14]:
vocab_size=5000

**One Hot Representation**

In [15]:
messages=X.copy()

In [16]:
messages.reset_index(inplace=True)

In [17]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vaibhav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [22]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [23]:
onehot=[one_hot(words,vocab_size) for words in corpus]

In [24]:
onehot[0]

[4922, 970, 776, 756, 109, 4856, 4819, 2018, 402, 3384]

**Embedded Representation**

In [25]:
sentence_length=20
embedded=pad_sequences(onehot,padding='pre',maxlen=sentence_length)

In [26]:
embedded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4922,
        970,  776,  756,  109, 4856, 4819, 2018,  402, 3384])

In [27]:
embedding_vector=40

**Creating Model**

In [28]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector,input_length=sentence_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


**Adding Dropout**

In [48]:
from tensorflow.keras.layers import Dropout
model1=Sequential()
model1.add(Embedding(vocab_size,embedding_vector,input_length=sentence_length))
model1.add(Dropout(0.3))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [49]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [30]:
len(embedded)

18285

In [31]:
y.shape

(18285,)

In [32]:
import numpy as np

In [33]:
X_final=np.array(embedded)
y_final=np.array(y)

In [34]:
X_final.shape

(18285, 20)

In [35]:
y_final.shape

(18285,)

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [37]:
X_train.shape

(12250, 20)

In [38]:
y_train.shape

(12250,)

In [39]:
X_test.shape

(6035, 20)

In [40]:
y_test.shape

(6035,)

In [41]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
12250/12250 [==============================] - 12s 1ms/sample - loss: 0.3097 - acc: 0.8540 - val_loss: 0.2004 - val_acc: 0.9167
Epoch 2/10
12250/12250 [==============================] - 10s 780us/sample - loss: 0.1387 - acc: 0.9473 - val_loss: 0.1970 - val_acc: 0.9200
Epoch 3/10
12250/12250 [==============================] - 12s 1ms/sample - loss: 0.0929 - acc: 0.9656 - val_loss: 0.2286 - val_acc: 0.9231
Epoch 4/10
12250/12250 [==============================] - 11s 883us/sample - loss: 0.0637 - acc: 0.9773 - val_loss: 0.2665 - val_acc: 0.9152
Epoch 5/10
12250/12250 [==============================] - 12s 979us/sample - loss: 0.0417 - acc: 0.9853 - val_loss: 0.3166 - val_acc: 0.9082
Epoch 6/10
12250/12250 [==============================] - 12s 991us/sample - loss: 0.0291 - acc: 0.9906 - val_loss: 0.4025 - val_acc: 0.9123
Epoch 7/10
12250/12250 [==============================] - 13s

In [50]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 25s 2ms/sample - loss: 0.3359 - acc: 0.8394 - val_loss: 0.2040 - val_acc: 0.9075
Epoch 2/10
12250/12250 [==============================] - 23s 2ms/sample - loss: 0.1505 - acc: 0.9403 - val_loss: 0.2065 - val_acc: 0.9196
Epoch 3/10
12250/12250 [==============================] - 20s 2ms/sample - loss: 0.1111 - acc: 0.9594 - val_loss: 0.2124 - val_acc: 0.9208
Epoch 4/10
12250/12250 [==============================] - 21s 2ms/sample - loss: 0.0844 - acc: 0.9706 - val_loss: 0.2584 - val_acc: 0.9163
Epoch 5/10
12250/12250 [==============================] - 20s 2ms/sample - loss: 0.0634 - acc: 0.9787 - val_loss: 0.3092 - val_acc: 0.9176
Epoch 6/10
12250/12250 [==============================] - 18s 1ms/sample - loss: 0.0527 - acc: 0.9819 - val_loss: 0.3149 - val_acc: 0.9133
Epoch 7/10
12250/12250 [==============================] - 22s 2ms/sample - loss: 0.0351 - acc: 0.9885 - val_loss: 0.3

**Performance Metrics and Accuracy**

In [42]:
y_pred=model.predict_classes(X_test)

In [43]:
y_pred[0]

array([1])

In [44]:
from sklearn.metrics import confusion_matrix

In [45]:
confusion_matrix(y_test,y_pred)

array([[3161,  258],
       [ 293, 2323]], dtype=int64)

In [46]:
from sklearn.metrics import accuracy_score

In [47]:
accuracy_score(y_test,y_pred)

0.9086992543496272

In [51]:
y_pred1=model1.predict_classes(X_test)

In [52]:
y_pred1[0]

array([1])

In [53]:
confusion_matrix(y_test,y_pred1)

array([[3100,  319],
       [ 217, 2399]], dtype=int64)

In [54]:
accuracy_score(y_test,y_pred1)

0.9111847555923778